In [1]:
!pip install neo4j langchain-community


  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s  0:00:00
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.4.43
    Uninstalling langsmith-0.4.43:
      Successfully uninstalled langsmith-0.4.43
  Attempting uninstall: langchain-core━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [langsmith]
    Found existing installation: langchain-core 1.0.7━━━━━━━━━ 1/4 [langsmith]
    Uninstalling langchain-core-1.0.7:━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [langsmith]
      Successfully uninstalled langchain-core-1.0.7━━━━━━━━━━━ 1/4 [langsmith]
  Attempting uninstall: langchain╺━━━━━━━━━━━━━━━━━━━ 2/4 [langchain-core]
    Found existing installation: langchain 1.0.8━━━━━━━━━ 3/4 [langchain]]
    Uninstalling langchain-1.0.8

In [20]:
import pandas as pd 

df = pd.read_csv('ogloszenia_warszawa_detailed.csv')
print(df.columns)
# Mapowanie nowych kolumn na stare nazwy, aby zachować interfejs

# Dodanie kolumny currency (PLN, bo cena jest w złotych)
df['currency'] = 'PLN'
# Reorganizacja kolumn do oryginalnej kolejności
df.head()

Index(['locality', 'street', 'rooms', 'area', 'price_total_zl', 'price_sqm_zl',
       'owner_type', 'date_posted', 'photo_count', 'url', 'image_url',
       'city_district', 'full_address', 'floor', 'year_built', 'building_type',
       'price_per_sqm_detailed', 'description_text', 'has_basement',
       'has_parking', 'kitchen_type', 'window_type', 'ownership_type',
       'equipment', 'latitude', 'longitude'],
      dtype='object')


,locality,street,rooms,area,price_total_zl,price_sqm_zl,owner_type,date_posted,photo_count,url,...,description_text,has_basement,has_parking,kitchen_type,window_type,ownership_type,equipment,latitude,longitude,currency
0,Warszawa Bielany,ul. Stefana Żeromskiego,3.0,50.0,1100000,22000.0,Bez pośredników,5 dni temu,19.0,https://adresowo.pl/o/mieszkanie-warszawa-biel...,...,Sprzedam 3 pokojowe mieszkanie z ogródkiem na ...,NaN,tak,aneks,plastikowe,własność,"Wyposażenie: internet, klimatyzacja, kuchenka,...",52.273962,20.963902,PLN
1,Warszawa Wola,ul. Góralska,2.0,37.0,719000,19359.0,Bez pośredników,wczorajnowe,9.0,https://adresowo.pl/o/mieszkanie-warszawa-wola...,...,BEZPOŚREDNIO Na sprzedaż: 2-pokojowe mieszkani...,tak,NaN,osobna,plastikowe,spółdzielcze własnościowe,NaN,52.237976,20.938676,PLN
2,Warszawa Bemowo,ul. Widawska,2.0,47.0,740000,15745.0,Bez pośredników,wczorajnowe,4.0,https://adresowo.pl/o/mieszkanie-warszawa-bemo...,...,"Szukasz mieszkania, które możesz urządzić po s...",tak,tak,osobna,plastikowe,spółdzielcze własnościowe,"Wyposażenie: lodówka, pralka, zmywarka",52.252781,20.928719,PLN
3,NaN,NaN,NaN,NaN,NaN,NaN,Pośrednik,NaN,NaN,https://adresowo.pl/mieszkania/warszawa/bemowo-g/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PLN
4,Warszawa Włochy,ul. Trzcinowa,2.0,49.0,zapytaj o cenę,NaN,Bez pośredników,NaN,7.0,https://adresowo.pl/o/mieszkanie-warszawa-wloc...,...,"Nowe mieszkanie 2-pokojowe o powierzchni 49,15...",NaN,NaN,NaN,NaN,NaN,Wyposażenie: internet,52.212363,20.919162,PLN


In [23]:
df = df.dropna()

In [24]:
def load_offer(tx, row):
    tx.run("""
    MERGE (o:Offer {url: $url})
    SET 
        o.rooms = $rooms,
        o.area = $area,
        o.price_total = $price_total,
        o.price_sqm = $price_sqm,
        o.owner_type = $owner_type,
        o.date_posted = $date_posted,
        o.photo_count = $photo_count,
        o.image_url = $image_url,
        o.floor = $floor,
        o.year_built = $year_built,
        o.price_per_sqm_detailed = $price_per_sqm_detailed,
        o.description = $description,
        o.has_basement = $has_basement,
        o.has_parking = $has_parking,
        o.kitchen_type = $kitchen_type,
        o.window_type = $window_type,
        o.ownership_type = $ownership_type,
        o.equipment = $equipment

    MERGE (loc:Location {full_address: $full_address})
    SET loc.locality = $locality,
        loc.street = $street,
        loc.city_district = $city_district

    MERGE (o)-[:LOCATED_AT]->(loc)

    MERGE (coord:Coordinates {lat: $lat, lon: $lon})
    MERGE (o)-[:HAS_COORDINATES]->(coord)

    MERGE (b:Building {building_type: $building_type})
    MERGE (o)-[:IN_BUILDING]->(b)
    """,
    url=row["url"],
    rooms=row["rooms"],
    area=row["area"],
    price_total=row["price_total_zl"],
    price_sqm=row["price_sqm_zl"],
    owner_type=row["owner_type"],
    date_posted=row["date_posted"],
    photo_count=row["photo_count"],
    image_url=row["image_url"],
    full_address=row["full_address"],
    locality=row["locality"],
    street=row["street"],
    city_district=row["city_district"],
    floor=row["floor"],
    year_built=row["year_built"],
    building_type=row["building_type"],
    price_per_sqm_detailed=row["price_per_sqm_detailed"],
    description=row["description_text"],
    has_basement=row["has_basement"],
    has_parking=row["has_parking"],
    kitchen_type=row["kitchen_type"],
    window_type=row["window_type"],
    ownership_type=row["ownership_type"],
    equipment=row["equipment"],
    lat=row["latitude"],
    lon=row["longitude"]
)

In [25]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    "bolt://localhost:7687",
    auth=("neo4j", "test123123")
)

In [26]:
with driver.session() as session:
    for i, row in df.iterrows():
        session.execute_write(load_offer, row)

In [28]:
df

,locality,street,rooms,area,price_total_zl,price_sqm_zl,owner_type,date_posted,photo_count,url,...,description_text,has_basement,has_parking,kitchen_type,window_type,ownership_type,equipment,latitude,longitude,currency
2,Warszawa Bemowo,ul. Widawska,2.0,47.0,740000,15745.0,Bez pośredników,wczorajnowe,4.0,https://adresowo.pl/o/mieszkanie-warszawa-bemo...,...,"Szukasz mieszkania, które możesz urządzić po s...",tak,tak,osobna,plastikowe,spółdzielcze własnościowe,"Wyposażenie: lodówka, pralka, zmywarka",52.252781,20.928719,PLN
22,Warszawa Włochy,ul. Wyrzyska,3.0,61.0,1075000,17623.0,Bez pośredników,3 dni temu,21.0,https://adresowo.pl/o/mieszkanie-warszawa-wloc...,...,Przestronne / Gotowe do wprowadzenia się 3 pok...,tak,tak,aneks,drewniane,własność,"Wyposażenie: internet, kuchenka, lodówka, piek...",52.205195,20.927790,PLN
85,Warszawa Targówek,ul. Głębocka,3.0,68.0,1390000,20441.0,Bez pośredników,4 dni temu,14.0,https://adresowo.pl/o/mieszkanie-warszawa-targ...,...,Prywatne ogłoszenie. Na sprzedaż: wyjątkowe mi...,tak,tak,osobna,plastikowe,własność,"Wyposażenie: internet, kuchenka, lodówka, mikr...",52.317798,21.068320,PLN
89,Warszawa Nowe Włochy,ul. Obywatelska,2.0,41.0,749000,18268.0,Bez pośredników,4 dni temu,10.0,https://adresowo.pl/o/mieszkanie-warszawa-nowe...,...,**** Nie współpracuję z agentami nieruchomość ...,tak,tak,aneks,drewniane,własność,"Wyposażenie: internet, kuchenka, lodówka, piek...",52.206325,20.894596,PLN
192,Warszawa Praga-Południe,ul. Terespolska,2.0,38.0,703000,18500.0,Bez pośredników,4 dni temu,14.0,https://adresowo.pl/o/mieszkanie-warszawa-prag...,...,Sprzedajemy nasze przytulne i zadbane mieszkan...,tak,tak,aneks,plastikowe,własność,"Wyposażenie: kuchenka, lodówka, mikrofalówka, ...",52.249353,21.070401,PLN
268,Warszawa Śródmieście,ul. Elektoralna,2.0,36.0,850000,23546.0,Bez pośredników,6 dni temu,19.0,https://adresowo.pl/o/mieszkanie-warszawa-srod...,...,Zapraszam do zakupu unikatowej nieruchomości w...,tak,tak,aneks,plastikowe,własność,"Wyposażenie: kuchenka, lodówka, piekarnik, pra...",52.240695,20.996900,PLN
277,Warszawa Bielany,ul. Hansa Christiana Andersena,2.0,42.0,730000,17381.0,Bez pośredników,ponad tydzień temu,15.0,https://adresowo.pl/o/mieszkanie-warszawa-biel...,...,Na sprzedaż dwupokojowe mieszkanie o powierzch...,tak,tak,osobna,plastikowe,spółdzielcze własnościowe,"Wyposażenie: kuchenka, lodówka, mikrofalówka, ...",52.279407,20.937066,PLN
278,Warszawa Wesoła,ul. Warszawska,3.0,67.0,970000,14510.0,Bez pośredników,ponad tydzień temu,17.0,https://adresowo.pl/o/mieszkanie-warszawa-weso...,...,Pośrednikom dziękuję za współpracę. Sprzedam b...,tak,tak,aneks,plastikowe,własność,"Wyposażenie: internet, klimatyzacja, kuchenka,...",52.245562,21.188360,PLN
293,Warszawa Bielany,ul. Hansa Christiana Andersena,3.0,74.0,1490000,20135.0,Bez pośredników,ponad tydzień temu,12.0,https://adresowo.pl/o/mieszkanie-warszawa-biel...,...,Sprzedam wygodne i wyjątkowo ciche mieszkanie ...,tak,tak,osobna,plastikowe,spółdzielcze własnościowe,"Wyposażenie: internet, kuchenka, lodówka, mikr...",52.282236,20.937357,PLN
399,Warszawa Praga-Południe,ul. Szaserów,3.0,48.0,715000,14896.0,Bez pośredników,ponad tydzień temu,13.0,https://adresowo.pl/o/mieszkanie-warszawa-prag...,...,"🌇 3 pokoje | 48,5 m² | Praga Południe, ul. Sza...",tak,tak,osobna,plastikowe,spółdzielcze własnościowe,"Wyposażenie: internet, kuchenka, lodówka, mikr...",52.229676,21.012229,PLN


In [29]:
query = """
MATCH (o:Offer)-[:LOCATED_AT]->(l:Location)
WHERE o.rooms = 2.0 AND l.city_district CONTAINS "Mokotów"
RETURN o.url, o.price_total AS price, o.area, l.full_address
ORDER BY price ASC
LIMIT 10;
"""

with driver.session() as session:
    results = session.run(query).data()
results

[{'o.url': 'https://adresowo.pl/o/mieszkanie-warszawa-mokotow-ul-stanislawa-pyjasa-2-pokojowe-q1t8y7',
  'price': '589000',
  'o.area': 32.0,
  'l.full_address': 'ul. Stanisława Pyjasa'},
 {'o.url': 'https://adresowo.pl/o/mieszkanie-warszawa-mokotow-ul-iberyjska-2-pokojowe-i8y0b3',
  'price': '795000',
  'o.area': 44.0,
  'l.full_address': 'ul. Iberyjska'}]

In [ ]:
#https://docs.langchain.com/oss/python/integrations/vectorstores/neo4jvector

In [ ]:
# Użycie zapytania Cypher przez LangChain Graph
query_cypher = """
MATCH (o:Offer)-[:LOCATED_AT]->(l:Location)
WHERE o.rooms = 2.0 AND l.city_district CONTAINS "Mokotów"
RETURN o.url, o.price_total AS price, o.area, l.full_address
ORDER BY price ASC
LIMIT 10;
"""

# Wykonanie zapytania przez LangChain
results_langchain = graph.query(query_cypher)
results_langchain


In [38]:
!pip uninstall -y neo4j


Found existing installation: neo4j 5.28.2
Uninstalling neo4j-5.28.2:
  Successfully uninstalled neo4j-5.28.2
